In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.8
        self.atr_period = 14
        self.atr_multiplier = 2 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = True
        self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size / 100000 # If we are trading AUDJPY, the value will be 0.01/100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        trend_SMA = df['trend_SMA']
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        if self.trend_ema10_sma50 == True:  ###########  use SMA50 and EMA10
            current_trend = curr['trend_SMA']
            prev_trend    = prev['trend_SMA']
        else: 
            current_trend = curr['trend'] ############ use EMA120 and EMA30
            prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bearish trend, a crossover early signal emerges when EMA30 turns upward, while EMA120 is still trending downward or flat.
                    
                    if current_slope_30 > 0 and current_slope_120 < 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bullish trend, a crossover early signal emerges when EMA30 turns downward, while EMA120 is still trending upward or flat.
                    
                    if current_slope_30 < 0 and current_slope_120 > 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """      
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    ALPHA_10 = 2 / 11
    MIN_BARS  = 120 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","slope30","slope120", 'SMA_50','trend_SMA',"EMA_10"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "slope30": NFLOAT,
                "slope120": NFLOAT,
                'SMA_50': NFLOAT,
                "trend_SMA": "category",
                "EMA_10": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")
            self.evt.consoleLog(f'{base_currency}USD, {fx_base}')
        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)
            # print out result to console log
            self.evt.consoleLog(f'USD{quote_currency}, {fx}')
        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,fx=fx,fx_base = fx_base, ts=self.last_time)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = ema10 = cl
            slope30 = slope120 = 0
        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = slope120 = 0
        else:
            prev  = self.data.iloc[n - 3] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = ema30 - prev["EMA_30"]
            slope120 = ema120 - prev["EMA_120"]
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope
        bar['slope30'] = slope30
        bar['slope120'] = slope120
        bar["EMA_10"]  = ema10
        
        # ---------- SMA 50 ---------------------------------------------
        if n >= 49:                                    # we already have 49 closes;
            sma50 = (self.data["Close"].iloc[-49:].sum() + cl) / 50
        else:
            current_mean = (self.data["Close"].sum() + cl) / (n + 1)
            weight = (n + 1) / 50            # grows from 0.02 … 0.98
            sma50 = weight * current_mean + (1 - weight) * cl
    
        bar["SMA_50"] = sma50
        
        # ---------- trend_SMA (EMA30 vs SMA50) --------------------------
        r2 = ema10 / sma50
        bar["trend_SMA"] = ("BU" if r2 > 1.001 else
                            "BE" if r2 < 0.999 else
                            "NO")
        

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])
        


        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass


#### Strategy 2:

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.tradeID = None
        
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.7
        self.atr_period = 14
        self.atr_multiplier = 1.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = True
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base


        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size / 100000 # If we are trading AUDJPY, the value will be 0.01/100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
    def _calculate_atr(self, atr_period = None, entry_point = None, trend = "NO"):
        
        atr_period = self.atr_period
        df = self.df
        atr_multiplier = self.atr_multiplier
        R = self.risk_reward_ratio
        
        current_idx = df.index.get_loc(entry_point)
        
        start_idx = max(0, current_idx - atr_period)
        
        df_slice = df.iloc[start_idx:current_idx + 1].copy()
        
        df_slice['H-L'] = df_slice['High'] - df_slice['Low']
        df_slice['H-PC'] = (df_slice['High'] - df_slice['Close'].shift(1)).abs()
        df_slice['L-PC'] = (df_slice['Low'] - df_slice['Close'].shift(1)).abs()
        df_slice['TR'] = df_slice[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        df_slice['ATR'] = df_slice['TR'].rolling(window=period).mean()
        
        atr_value = df_slice['ATR'].iloc[-1]
        
        pip_size = atr_multiplier * atr_value
        
        current_close_price = df.loc[entry_point]["Close"]
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE":
            stop_loss_price = current_close_price + pip_size
            take_profit_price = current_close_price - R * pip_size
        else:  # bullish
            stop_loss_price = current_close_price - pip_size
            take_profit_price = current_close_price + R * pip_size
        
        return stop_loss_price, take_profit_price
        
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts, trend):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do
        
        df = self.df
        if self.enter_market_trade_side == "BU":          # long trade
            
            if (df.loc[ts]["EMA_5"] < df.loc[ts]["EMA_50"]) or (df.loc[ts]["MACD_fast"] < 0):
                    current_price = df.loc[ts]["Close"]
                    
                    self.evt.update_opened_order(tradeID=self.tradeID, tp=current_price, sl=current_price)
                    
                    # close trade
                    fsm.reset()
        if self.enter_market_trade_side == "BE":          # long trade
            
            if (df.loc[ts]["EMA_5"] > df.loc[ts]["EMA_50"]) or (df.loc[ts]["MACD_fast"] > 0):
                    current_price = df.loc[ts]["Close"]
                    # close trade
                    self.evt.update_opened_order(tradeID=self.tradeID, tp=current_price, sl=current_price)
                    fsm.reset()
                    
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        df = self.df
        curr   = df.loc[ts]
        current_trend = curr['trend'] ############ use EMA5 and EMA50 and MACD-fast
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
                # selects the row in df where the index = ts
        if self.position_open:
            self.monitor_exit(ts, current_trend)        # <-- will emit console logs
            return                       # <-- early exit
        
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        prev_trend    = prev['trend']
        
        
        
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================
        
        if (current_trend == "BE"):
            
            stop, take = self._calculate_atr(self, atr_period = 14, entry_point = ts, trend = current_trend)
                            
            # Logic to calculate lot_size
            entry_price = closes.loc[ts]
            stop_loss_pips = abs(stop - entry_price) / self.pip_size
            lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
            # Log for debug
            self.evt.consoleLog(
                f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                f"SL={stop:.5f} TP={take:.5f}"
            )
            
            # place selling order
            self.send_entry_order(ts, trend="BE",
                        entry_price=entry_price,
                        stop=stop, take=take,
                        volume=lot_size)
                        
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================                       
                                
        if (current_trend == "BU"):
            
            stop, take = self._calculate_atr(self, atr_period = 14, entry_point = ts, trend = current_trend)
                            
            # Logic to calculate lot_size
            entry_price = closes.loc[ts]
            stop_loss_pips = abs(stop - entry_price) / self.pip_size
            lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
            # Log for debug
            self.evt.consoleLog(
                f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                f"SL={stop:.5f} TP={take:.5f}"
            )
            
            # place selling order
            self.send_entry_order(ts, trend="BU",
                        entry_price=entry_price,
                        stop=stop, take=take,
                        volume=lot_size)

    def reset(self):
        self.position_open = False
        self.stop_loss_price = None
        self.take_profit_price = None
        self.tradeID = None
        self.enter_market_trade_side = None

class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    MIN_BARS  = 50 
    MAX_BARS  = 300
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
            "Datetime", "Open", "High", "Low", "Close",
            "Green", "Instrument", "EMA_5", "EMA_50", "EMA_12", 'EMA_26',"MACD_slow","MACD_fast", "trend"
        ])
        .astype({
            "Datetime": "datetime64[ns]",
            "Open":  NFLOAT,
            "High":  NFLOAT,
            "Low":   NFLOAT,
            "Close": NFLOAT,
            "Green": "category",
            "Instrument": "object",
            "EMA_5": NFLOAT,
            "EMA_50": NFLOAT,
            "EMA_12": NFLOAT,
            "EMA_26": NFLOAT,
            "MACD_slow": NFLOAT,
            "MACD_fast": NFLOAT,
            "trend": "category",
        })
        .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")
            self.evt.consoleLog(f'{base_currency}USD, {fx_base}')
        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)
            # print out result to console log
            self.evt.consoleLog(f'USD{quote_currency}, {fx}')
        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,fx=fx,fx_base = fx_base, ts=self.last_time)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
            
            
    def _aggregate_15m(self, last3):
        n = len(self.data)
        op = last3["Open"].iloc[0]
        cl = last3["Close"].iloc[-1]
        
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }
        ALPHA_macd_fast = 2/19 #################
        if n == 0:
            ema5 = ema50 = ema12 = ema26 = cl
            macd_slow = 0
            macd_fast = 0
        else:
            prev = self.data.iloc[n - 1]
            ema5 = (1 - (2 / 6))  * prev["EMA_5"]  + (2 / 6)  * cl  # EMA5
            ema50 = (1 - (2 / 51)) * prev["EMA_50"] + (2 / 51) * cl  # EMA50
    
            ema12 = (2/13) * cl + (2/13) * prev["EMA_12"]
            ema26 = (2/27) * cl + (2/27) * prev["EMA_26"]
            macd_slow = ema12 - ema26
            macd_fast =  (1 - (2/19))  * prev["macd_fast"]  + (2/19)  * macd_slow
        
        # Assign indicators
        bar["EMA_5"] = ema5
        bar["EMA_50"] = ema50
        bar["EMA_12"] = ema12
        bar["EMA_26"] = ema26
        bar["MACD_slow"] = macd_slow
        bar["MACD_fast"] = macd_fast
    
        # Determine trend
        if ema5 > ema50 and macd_fast > 0:
            bar["trend"] = "BU"
        elif ema5 < ema50 and macd_fast < 0:
            bar["trend"] = "BE"
        else:
            bar["trend"] = "NO"
    
        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime", drop=False)])
        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.tradeID        = fsm.pending["tradeID"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear

        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None

                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass





